In [14]:
from azure.identity import AzureCliCredential
from azure.ai.projects import AIProjectClient

import os
from dotenv import load_dotenv
load_dotenv()

ENDPOINT = os.getenv("ENDPOINT")
subscription_key = os.getenv("SUBSCRIPTION_KEY")


In [15]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint= ENDPOINT,
    api_key=subscription_key,
)


In [19]:
model_name = "gpt-4o"
deployment = "gpt-4o"
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=ENDPOINT,
    api_key=subscription_key,
)

response = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "You are a comedian."},
        {"role": "user", "content": "Tell me a joke about AI."}
    ],
    max_tokens=100,
    temperature=0.7,
    top_p=0.95,
    model=deployment
)

print(response.choices[0].message.content)

Sure! 

Why did the AI go broke?  
Because it kept investing in binary options!
